# Paul Zikopoulos Week 3 Peer-Graded Assignment

## Setting up the environment

In [2]:
# Only need to run once for each to install
#!pip install folium
#!pip install geocoder

     |████████████████████████████████| 92kB 14.9MB/s eta 0:00:01
     |████████████████████████████████| 102kB 13.7MB/s ta 0:00:01


In [13]:
# Import the libraries I will use
import pandas as pd
from bs4 import BeautifulSoup
import requests
import folium
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# The Python Geocode
import geocoder
import types
from botocore.client import Config
import ibm_boto3

## Getting the data and pre-processing to match the required outcome

In [5]:
# Scrape the https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M Wiki page to get the data that is in the table of postal codes
# This is the Beautiful Soup method that we don't have to use but it is easier.

cdn_postal_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
PZ_soup = BeautifulSoup(cdn_postal_url,'lxml')

columns = ['Postcode','Borough','Neighbourhood']
values = []

for items in PZ_soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    values.append([data[0].text, data[1].text, data[2].text[0:-1]])


In [6]:
# Now put this data into a Pandas data frame
cdn_postal_code_table = pd.read_html(cdn_postal_url)

In [7]:
# Look at the data
cdn_postal_code_table[0].head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [29]:
# Only process the cells that have an assigned borough. 
# Ignore cells with a borough that is Not assigned.
# Delete all rows with column 'Age' has value 30 to 40 
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
# More than one neighborhood can exist in one postal code area, they will be separated by commas (Index position 11 should show a , and multiple Neighbourhoods)
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. For index position 9
# Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

# Build out the data frame
PZ_data = pd.DataFrame(values, columns=columns)

# Get rid of not assigned columns
PZ_data = PZ_data[PZ_data.Borough != 'Not assigned']

# Build out the data we need with a for loop
for x in range(PZ_data.shape[0]):
    if PZ_data.iloc[x][2] == 'Not assigned':
        PZ_data.iloc[x][2] = PZ_data.iloc[x][1]

# Group by the order specified
# I needed assistance for this step
PZ_data = PZ_data.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
PZ_data.head(11)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [10]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
PZ_data.shape

(103, 3)

In [ ]:
# In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.
# We will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.
# We need to get the LAT and LONG of the Candian Postal codes so we can plot locations on a map later on.

# Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, 
# here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [24]:
# The code was removed by Watson Studio for sharing.

In [53]:
LAT_LONG_body = client_c031707209a846b49a277a00b79d3c9f.get_object(Bucket='ibmcourserapythonandml-donotdelete-pr-o1pq0erjjblno9',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

LAT_LONG_coordinates = pd.read_csv(LAT_LONG_body)
LAT_LONG_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [55]:
# Let's get the cordinates
# If you wanted to read this from disk you would use this line, but I read it from the IBM COS above.
# LAT_LONG_coordinates = pd.read_csv("Geospatial_Coordinates.csv")

LAT_LONG_coordinates.rename(columns={'Postcal Code':'PostalCode'}, inplace=True)
LAT_LONG_coordinates.head(5)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [60]:
# Let's talk the LAT LONG cordinates we just got and merge it with our Candian Postal Code data.
PZ_final_result = PZ_data.merge(LAT_LONG_coordinates, left_on='Postcode', right_on='PostalCode')
# Drop the duplicate column named PostalCode 
PZ_final_result.drop(['PostalCode'], axis=1, inplace=True)
PZ_final_result.head(11)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [61]:
# This is the GEOCODER code in the graded assignment instruction. But I found I didn't use it because it's not reliable.

# The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. 
# So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call 
# again and you would get the coordinates. 

# So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. 
# Taking postal code M5G as an example, your code would look something like this:

# Initialize your variable to None
#lat_lng_coords = None

# Loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

## Map out the Postal Codes acoss the GTA with neighborhood names

In [67]:
latitude, longitude = 43.653963, -79.387207
map_of_gta = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng,boro,neigh in zip(PZ_final_result.Latitude, PZ_final_result.Longitude, PZ_final_result.Borough, PZ_final_result.Neighbourhood):
    label = '{} - {}'.format(neigh, boro)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_of_gta)  
map_of_gta

## Map out the Postal Codes acoss the city for Toronto ONLY wth neighborhood names

In [70]:
latitude, longitude = 43.653963, -79.387207
map_of_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
PZ_final_result_city_of_toronto = PZ_final_result[PZ_final_result['Borough'].str.contains('Toronto')]

for lat,lng,boro,neigh in zip(PZ_final_result_city_of_toronto.Latitude, PZ_final_result_city_of_toronto.Longitude, PZ_final_result_city_of_toronto.Borough, PZ_final_result_city_of_toronto.Neighbourhood):
    label = '{} - {}'.format(neigh, boro)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_of_toronto)  
map_of_toronto